In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

import pandas as pd

In [2]:

df0 = pd.read_csv('Virginia.csv')
df1=df0[['county_name', 'jurisdiction', 'precinct', 'candidate', 'votes']]
df1.loc[(df1['candidate']!='Hillary Clinton') & (df1['candidate']!='Donald Trump'), 'candidate']='Other'


C:\Users\alasseter\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [3]:
df1.head(2)

,county_name,jurisdiction,precinct,candidate,votes
0,Accomack County,ACCOMACK COUNTY,# AB - Central Absentee Precinct,Hillary Clinton,784
1,Accomack County,ACCOMACK COUNTY,# AB - Central Absentee Precinct,Other,32


In [95]:
df2=df1.groupby(['county_name', 'jurisdiction', 'precinct', 'candidate']).sum()
df2.tail(6)

votes
county_name jurisdiction precinct     candidate             
York County YORK COUNTY  501 - TABB   Donald Trump      1705
                                      Hillary Clinton    955
                                      Other              179
                         502 - BETHEL Donald Trump      1496
                                      Hillary Clinton   1184
                                      Other              210

In [39]:
df3=df2.unstack(level=-1)
df3.head(5)

votes  \
candidate                                                        Donald Trump   
county_name     jurisdiction    precinct                                        
Accomack County ACCOMACK COUNTY # AB - Central Absentee Precinct          644   
                                ## Provisional                              1   
                                101 - CHINCOTEAGUE                       1188   
                                201 - ATLANTIC                            553   
                                202 - GREENBACKVILLE                      697   

                                                                                  \
candidate                                                        Hillary Clinton   
county_name     jurisdiction    precinct                                           
Accomack County ACCOMACK COUNTY # AB - Central Absentee Precinct             784   
                                ## Provisional                                 3   
                                101 - CHINCOTEAGUE                           444   
                                201 - ATLANTIC                               175   
                                202 - GREENBACKVILLE                         302   

                                                                        
candidate                                                        Other  
county_name     jurisdiction    precinct                                
Accomack County ACCOMACK COUNTY # AB - Central Absentee Precinct    59  
                                ## Provisional                       2  
                                101 - CHINCOTEAGUE                  48  
                                201 - ATLANTIC                      22  
                                202 - GREENBACKVILLE                31

In [42]:
df4=df3.reset_index()
df4.head()

county_name     jurisdiction                          precinct  \
candidate                                                                       
0          Accomack County  ACCOMACK COUNTY  # AB - Central Absentee Precinct   
1          Accomack County  ACCOMACK COUNTY                    ## Provisional   
2          Accomack County  ACCOMACK COUNTY                101 - CHINCOTEAGUE   
3          Accomack County  ACCOMACK COUNTY                    201 - ATLANTIC   
4          Accomack County  ACCOMACK COUNTY              202 - GREENBACKVILLE   

                 votes                        
candidate Donald Trump Hillary Clinton Other  
0                  644             784    59  
1                    1               3     2  
2                 1188             444    48  
3                  553             175    22  
4                  697             302    31

In [48]:
df4['votes'].columns

Index(['Donald Trump', 'Hillary Clinton', 'Other'], dtype='object', name='candidate')

In [46]:
df4['votes']['Donald Trump'][0]

644

In [97]:
df4['jurisdiction'][0]

'ACCOMACK COUNTY'

In [102]:
accomack=df4[df4['jurisdiction']=='ACCOMACK COUNTY']
accomack.head(5)

county_name     jurisdiction                          precinct  \
candidate                                                                       
0          Accomack County  ACCOMACK COUNTY  # AB - Central Absentee Precinct   
1          Accomack County  ACCOMACK COUNTY                    ## Provisional   
2          Accomack County  ACCOMACK COUNTY                101 - CHINCOTEAGUE   
3          Accomack County  ACCOMACK COUNTY                    201 - ATLANTIC   
4          Accomack County  ACCOMACK COUNTY              202 - GREENBACKVILLE   

                 votes                        
candidate Donald Trump Hillary Clinton Other  
0                  644             784    59  
1                    1               3     2  
2                 1188             444    48  
3                  553             175    22  
4                  697             302    31

In [111]:
precincts=list(accomack['precinct'].value_counts().index)
print(precincts[5])
precincts[:8]

202 - GREENBACKVILLE


['# AB - Central Absentee Precinct',
 '802 - MELFA',
 '701 - NANDUA',
 '## Provisional',
 '403 - SAXIS',
 '202 - GREENBACKVILLE',
 '803 - WACHAPREAGUE',
 '901 - PAINTER']

In [123]:
list(accomack['votes']['Donald Trump'])

[644,
 1,
 1188,
 553,
 697,
 592,
 342,
 489,
 153,
 243,
 119,
 637,
 221,
 976,
 369,
 392,
 349,
 618]

In [ ]:
color=['#122A7F','#008080','#92A5E8']

In [132]:
# Let's display that with plotly.

mydata1 = go.Bar(x=list(accomack['precinct'].value_counts().index), 
                 y=list(accomack['votes']['Donald Trump']), 
                 marker=dict(color='#122A7F'),
                 name='Trump')
mydata2 = go.Bar(x=list(accomack['precinct'].value_counts().index), 
                 y=list(accomack['votes']['Hillary Clinton']), 
                 marker=dict(color='#008080'),
                 name='Clinton')
mydata3 = go.Bar(x=list(accomack['precinct'].value_counts().index), 
                 y=list(accomack['votes']['Other']), 
                 marker=dict(color='#92A5E8'),
                 name='Other')

mylayout = go.Layout(
    title='Grouped bar chart',
    xaxis=dict(title='Candidates'),
    yaxis=dict(title='Number of Votes')    
)
fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
iplot(fig)

In [142]:
# Now turn that into a function.
def juris_picker(juris_name):
    juris_df=df4[df4['jurisdiction']==juris_name]
    
    mydata1 = go.Bar(x=list(juris_df['precinct'].value_counts().index), 
                     y=list(juris_df['votes']['Donald Trump']), 
                     marker=dict(color='#122A7F'),
                     name='Trump')
    mydata2 = go.Bar(x=list(juris_df['precinct'].value_counts().index), 
                     y=list(juris_df['votes']['Hillary Clinton']), 
                     marker=dict(color='#008080'),
                     name='Clinton')
    mydata3 = go.Bar(x=list(juris_df['precinct'].value_counts().index), 
                     y=list(juris_df['votes']['Other']), 
                     marker=dict(color='#92A5E8'),
                     name='Other')

    mylayout = go.Layout(
        title='Votes by candidate for: {}'.format(juris_name),
        xaxis=dict(title='Precincts'),
        yaxis=dict(title='Number of Votes')    
    )
    fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
    iplot(fig)

In [143]:
jurisdiction_name='YORK COUNTY'
juris_picker(jurisdiction_name)

In [144]:
jurisdiction_name='ACCOMACK COUNTY'
juris_picker(jurisdiction_name)